## Permission denied
We can create a q by creating a file in `/dev/mqueue`, but opening it results in Permission denied.

In [72]:
QFILE=`mktemp /dev/mqueue/mqtest.XXXX`

In [73]:
echo $QFILE

/dev/mqueue/mqtest.gVAC


In [74]:
ls -l $QFILE

-rw------- 1 jnewsome jnewsome 80 May  9 17:24 /dev/mqueue/mqtest.gVAC


In [75]:
cat $QFILE

QSIZE:0          NOTIFY:0     SIGNO:0     NOTIFY_PID:0     


In [76]:
QNAME="/$(basename $QFILE)"
echo $QNAME

/mqtest.gVAC


In [77]:
gcc -o mq_send -xc - -lrt <<EOF
#include <fcntl.h>
#include <sys/stat.h>
#include <mqueue.h>
#include <stdio.h>
#include <stdlib.h>

#define CHECK_GTE0(x) { \
  if ((x) < 0) {\
    perror(#x);\
    exit(EXIT_FAILURE);\
  }\
}

#define CHECK_EQ0(x) { \
  if ((x) != 0) {\
    perror(#x);\
    exit(EXIT_FAILURE);\
  }\
}

int main(int argc, char **argv) {
  mqd_t q;
  CHECK_GTE0(q = mq_open(argv[1], O_WRONLY));
  struct mq_attr attr;
  CHECK_EQ0(mq_getattr(q, &attr));
  printf("flags: %ld\n", attr.mq_flags);
  printf("maxmsg: %ld\n", attr.mq_maxmsg);
  printf("msgsize: %ld\n", attr.mq_msgsize);
  printf("curmsgs: %ld\n", attr.mq_curmsgs);
}
EOF

In [78]:
./mq_getattr $QNAME

flags: 0
maxmsg: 10
msgsize: 8192
curmsgs: 0


In [79]:
gcc -o mq_send -xc - -lrt <<EOF
#include <fcntl.h>
#include <sys/stat.h>
#include <mqueue.h>
#include <stdio.h>
#include <stdlib.h>

#define CHECK_GTE0(x) { \
  if ((x) < 0) {\
    perror(#x);\
    exit(EXIT_FAILURE);\
  }\
}

#define CHECK_EQ0(x) { \
  if ((x) != 0) {\
    perror(#x);\
    exit(EXIT_FAILURE);\
  }\
}

int main(int argc, char **argv) {
  mqd_t q;
  CHECK_GTE0(q = mq_open(argv[1], O_WRONLY));
  struct mq_attr attr;
  CHECK_EQ0(mq_getattr(q, &attr));
  
  char *buf = malloc(attr.mq_msgsize);
  size_t n;
  CHECK_GTE0(n = fread(buf, 1, attr.mq_msgsize, stdin));
  CHECK_GTE0(mq_send(q, buf, n, 0));
}
EOF

In [80]:
./mq_send $QNAME <<EOF
Hello message queue
Secondline
EOF

In [81]:
cat $QFILE

QSIZE:31         NOTIFY:0     SIGNO:0     NOTIFY_PID:0     


In [82]:
./mq_getattr $QNAME

flags: 0
maxmsg: 10
msgsize: 8192
curmsgs: 1


In [83]:
gcc -o mq_recv -xc - -lrt <<EOF
#include <fcntl.h>
#include <sys/stat.h>
#include <mqueue.h>
#include <stdio.h>
#include <stdlib.h>

#define CHECK_GTE0(x) { \
  if ((x) < 0) {\
    perror(#x);\
    exit(EXIT_FAILURE);\
  }\
}

#define CHECK_EQ0(x) { \
  if ((x) != 0) {\
    perror(#x);\
    exit(EXIT_FAILURE);\
  }\
}

int main(int argc, char **argv) {
  mqd_t q;
  CHECK_GTE0(q = mq_open(argv[1], O_RDONLY));
  struct mq_attr attr;
  CHECK_EQ0(mq_getattr(q, &attr));
  
  char *buf = malloc(attr.mq_msgsize);
  ssize_t n;
  int priority;
  CHECK_GTE0(n = mq_receive(q, buf, attr.mq_msgsize, &priority));
  CHECK_GTE0(fwrite(buf, 1, n, stdout));
}
EOF

In [84]:
./mq_recv $QNAME

Hello message queue
Secondline
